In [1]:
import pandas as pd
import numpy as np

1\. Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named "data_int.txt". Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named "data_float.txt". Use the `cat` command to print the content of the file.
+ load the txt file of the previous point and convert it to a csv file by hand.

In [19]:
list_int = np.arange(10)

textfile_integer = open("data/data_int.txt", "w")

for element in list_int:
    textfile_integer.write(str(element) + "\n")
textfile_integer.close()

!type data_int.txt #this is to replacement for cat command in windows

#also this works too:


# with open("data/data_int.txt", mode = 'r') as f:
#     # print-out the whole file
    
#     for line in f:
#         # print line by line
#         print(line)


The system cannot find the file specified.
Error occurred while processing: data_int.txt.
The system cannot find the file specified.
Error occurred while processing: #this.
The system cannot find the file specified.
Error occurred while processing: is.
The system cannot find the file specified.
Error occurred while processing: to.
The system cannot find the file specified.
Error occurred while processing: replacement.
The system cannot find the file specified.
Error occurred while processing: for.
The system cannot find the file specified.
Error occurred while processing: cat.
The system cannot find the file specified.
Error occurred while processing: command.
The system cannot find the file specified.
Error occurred while processing: in.
The system cannot find the file specified.
Error occurred while processing: windows.


In [8]:
list_float_5x5 = np.arange(1,26,dtype = float).reshape(5,5) #created 25 element array then make it 5x5

np.savetxt('data_float.txt',list_float_5x5) #saved it to txt


In [7]:
csv = open("data/data_float.csv", "w")


with open('data/data_float.csv', 'w', newline='') as file:
    for element in list_float_5x5:
        file.write(str(element) + ",")
    file.close()
    

pd.read_csv('data/data_float.csv')


,[1. 2. 3. 4. 5.],[ 6. 7. 8. 9. 10.],[11. 12. 13. 14. 15.],[16. 17. 18. 19. 20.],[21. 22. 23. 24. 25.],Unnamed: 5


2\. Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [5]:
import json # import the JSON module
data = json.load(open('data/user_data.json'))
with open('data/user_data.json') as f:
    data = json.load(f)
#data['ID']['JobTitle']['EmailAddress']['FirstNameLastName']['CreditCard']['CreditCardType'] #the colum names
data = pd.DataFrame(data)
new_DF = data[data['CreditCardType'] == 'American Express'] #filtered CreditCardType == American Express
new_DF.to_csv("Filtered_User_Data.csv")
new_DF

,ID,JobTitle,EmailAddress,FirstNameLastName,CreditCard,CreditCardType
1,2,Investment Advisor,Clint_Thorpe5003@bulaffy.com,Clint Thorpe,7083-8766-0251-2345,American Express
11,12,Retail Trainee,Phillip_Carpenter9505@famism.biz,Phillip Carpenter,3657-0088-0820-5247,American Express
27,28,Project Manager,Russel_Graves1378@extex.org,Russel Graves,6718-4818-8011-6024,American Express
38,39,Stockbroker,Leanne_Newton1268@typill.biz,Leanne Newton,5438-0816-4166-4847,American Express
56,57,Budget Analyst,Tony_Giles1960@iatim.tech,Tony Giles,8130-3425-7573-7745,American Express
61,62,CNC Operator,Owen_Allcott5125@bauros.biz,Owen Allcott,4156-0107-7210-2630,American Express
67,68,Project Manager,Liam_Lynn3280@kideod.biz,Liam Lynn,7152-3247-6053-2233,American Express
73,74,Dentist,Regina_Woodcock5820@yahoo.com,Regina Woodcock,0208-1753-3870-8002,American Express
80,81,HR Specialist,Carter_Wallace9614@atink.com,Carter Wallace,4256-7201-6717-4322,American Express
91,92,Staffing Consultant,Maia_Stark2797@jiman.org,Maia Stark,3851-1403-1734-6321,American Express


3\. Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [6]:
#data = pd.read_csv('https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv')
data = pd.read_csv('https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv?dl=1')
#get tokinize error so added '?dl=1' the end of the url to load it

data

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,3,2,4,0,5,0,0,0,11,...,2,5,5,0,1,1,4,0,1,2
8120,0,5,2,4,0,5,0,0,0,11,...,2,5,5,0,0,1,4,0,4,2
8121,0,2,2,4,0,5,0,0,0,5,...,2,5,5,0,1,1,4,0,1,2
8122,1,3,3,4,0,8,1,0,1,0,...,1,7,7,0,2,1,0,7,4,2


In [19]:
mean_group = data.groupby(['class']).mean()
print(mean_group)


       cap-shape  cap-surface  cap-color   bruises      odor  gill-attachment  \
class                                                                           
0       3.266160     1.615970   4.581749  0.653992  4.334601         0.954373   
1       3.436159     2.055158   4.421859  0.159346  3.940756         0.995403   

       gill-spacing  gill-size  gill-color  stalk-shape  ...  \
class                                                    ...   
0          0.285171   0.068441    6.622624     0.615970  ...   
1          0.028601   0.567926    2.863636     0.514811  ...   

       stalk-surface-below-ring  stalk-color-above-ring  \
class                                                     
0                      1.798479                6.098859   
1                      1.394280                5.512768   

       stalk-color-below-ring  veil-type  veil-color  ring-number  ring-type  \
class                                                                          
0                    

4\. Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [57]:
binar = pd.read_csv("data/credit_card.dat")

def bin2int2char(n):  #input is string we have to slice it to 6 chars
    
    a = str() #empty string; we are going to append new chars to it
    for i in range(0,len(str(n))-6,6): #this returns i = [0,6,12,...114] IT AUTOMATICALLY IGNORES LAST 4 DIGIT
        charr = chr(int(n[i:i+6],2))  #takes the string slices it, makes it binary to dec
                                        #and converts that val to char
        a+=charr #adds char to the string a
    return a

#make it dataframe



binar[binar.columns[0]] = binar[binar.columns[0]].apply(bin2int2char) #aplying the function to every row


#Rename the column 
binar.rename(columns={binar.columns[0]:bin2int2char(binar.columns[0])})


,7648 5673 3775 2271
0,3257 8247 3354 2266
1,2722 0001 4011 6652
2,0661 3063 3742 3150
3,0432 1608 1462 4742
4,5827 2027 8785 7303
5,5774 8528 2087 1117
6,8140 1210 6352 2845
7,5764 1133 7301 7100
8,6456 1737 4126 6726
9,1228 8631 7382 0000


In [29]:
#manual csv 
#testing

binar2 = pd.read_csv("data/credit_card.dat")

testval = binar2[binar2.columns[0]][0]
bin2int2char(testval)
print((bin2int2char(testval)))

3257 8247 3354 2266


5\. **Optional**: load the remote file:

- https://www.dropbox.com/s/aamg1apjhclecka/regression_generated.csv

with Pandas and create a scatter plot with all possible combinations of the following features:
    
  + features_1
  + features_2
  + features_3